# GPT Prompt 생성


> **주제: 한국 보험 약관의 해석 및 비판적 분석** <br/><br/>
Topic: Interpretation and critical analysis of Korean insurance terms





---



> *약관의 '깊이 있는 이해와 논리적 분석' (CoT) 과 '다양한 문제점 및 해결책 발상' (ToT)*

1.  약관 문구 '심층 이해' (CoT 집중): 주어진 약관 문구 CoT통해 단계별 분석, 글자 그대로의 의미, 핵심 용어, 적용 조건 등 정확히 파악(ToT의 첫 '생각'이 심층 이해)
2. 잠재적 '문제점 발상' 및 근거 제시 (ToT 발상 + CoT 근거): 1단계에서 이해한 내용을 바탕으로, 이 약관이 소비자에게 어떤 잠재적인 문제(오해, 불이익, 허점 등)일으킬 수 있는지 여러 가지 아이디어 발상. 각 문제점 아이디어를 낼 때, 왜 그게 문제라고 생각했는지 약관 내용에 기반해서 CoT처럼 논리적으로 근거 제시(ToT 발상 과정 안에 CoT 추론 포함)
3. 문제점 '평가' 및 중요 문제 선정 (ToT 평가): 발상된 여러 문제점들을 심각성, 발생 가능성, 모호성 등의 기준으로 평가해서, 가장 중요하고 시급한 문제점 하나(또는 몇 개) 선택
4. '개선 방안 발상' 및 효과 설명 (ToT 발상 + CoT 효과): 3단계에서 선택된 핵심 문제점을 해결하기 위한 여러 가지 개선 방안(약관 문구 수정, 추가 설명, 절차 개선 등) 아이디어 발상. 각 개선 방안 아이디어 낼 때, 왜 이 방안이 문제를 해결할 수 있는지 CoT처럼 논리적으로 설명하게 함. (ToT 발상 과정 안에 CoT 설명 포함)
5. 개선 방안 '평가' 및 최적 방안 선정 (ToT 평가): 발상된 개선 방안들을 효과성, 실현 가능성, 보험사/소비자 영향 등의 기준으로 평가, 가장 현실적이고 효과적인 최적 방안을 선택



---



## Basic Setting

In [2]:
import openai
import time
import os
from google.colab import drive
drive.mount('/content/drive')
!pip install streamlit
import streamlit as st

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [3]:
client = openai.OpenAI(api_key = xx)

In [4]:
from collections import defaultdict
import re
import json
!pip install PyMuPDF
import fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 36.6 MB/s eta 0:00:00


## 1. 약관 내용 심층 이해

### 약관 내용을 구조별로 정리
- 약관 파일 읽음
- 약관 내용 구조별로 나누기
- 각 구조의 주요 내용을 계약자가 이해하기 쉬운 말로 바꾸기
- 어떻게 쉬운 말로 바꾸었는지 보여주기
- self-consistency로 여러 번 시도하여 가장 잘 정리된 결과 선택
- 결과 구조화, 혹은 표로 시각화

### 약관 pdf파일에서 텍스트 추출

In [5]:
import fitz  # PyMuPDF 임포트

def extract_text_from_pdf(pdf_path):
    """PDF 파일 경로를 받아 텍스트를 추출하여 반환"""
    text = ""
    try:
        doc = fitz.open(pdf_path)
        print(f"PDF 파일 로드 성공: {pdf_path}, 총 {doc.page_count} 페이지")
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text()

        doc.close()
        print("PDF 텍스트 추출 완료.")
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {pdf_path}")
        return None
    except Exception as e:
        print(f"PDF 텍스트 추출 중 오류 발생: {e}")
        return None
    text = text.replace('\t', '').replace('\n', '').replace('\r', '')
    return text


In [ ]:
pdf_file_path = '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/무배당 삼성화재 다이렉트보험약관.pdf'


extracted_text = extract_text_from_pdf(pdf_file_path)
if extracted_text:
    print(f"추출된 텍스트 일부:\n{extracted_text[:500]}...")
else:
    print("텍스트 추출 실패.")
#     # 에러 처리: 텍스트 추출 실패 시 분석 진행 불가


PDF 텍스트 추출 중 오류 발생: no such file: '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/무배당 삼성화재 다이렉트보험약관.pdf'
텍스트 추출 실패.


#### 폴더 내 파일들 순회하며 pdf는 txt로 변환하고 txt는 그대로 두어  txt 파일 목록 반환

In [7]:
base_drive_folder_path = '/content/drive/MyDrive/PROJECT/cn_api_project/data'

In [6]:
# --- Google Drive 폴더 내 파일 처리 및 RAG 문서 목록 생성 함수 ---
def process_drive_folder_for_rag(base_drive_folder_path):
    """
    Google Drive의 특정 폴더를 탐색하며 PDF는 TXT로 변환하고,
    RAG 입력으로 사용할 모든 TXT 파일 경로 목록을 반환합니다.
    """
    txt_list = []

    print(f"--- Google Drive 폴더 탐색 및 파일 처리 시작: '{base_drive_folder_path}' ---")

    if not os.path.isdir(base_drive_folder_path):
        print(f"!!! 오류: 지정된 경로가 폴더가 아니거나 존재하지 않습니다 - '{base_drive_folder_path}' !!!")
        print("Google Drive 마운트 상태와 경로를 확인해주세요.")
        return [] # 오류 시 빈 리스트 반환


    # os.walk()를 사용하여 폴더 및 하위 폴더 순회
    for root, dirs, files in os.walk(base_drive_folder_path):
        print(f"\n폴더 탐색 중: {root}")

        for file in files:
            file_path = os.path.join(root, file)
            file_name, file_ext = os.path.splitext(file) # 파일 이름과 확장자 분리
            file_ext = file_ext.lower() # 확장자를 소문자로 변환하여 비교 (대소문자 구분 없이)

            # PDF 파일인 경우
            if file_ext == '.pdf':
                output_txt_path = os.path.join(root, file_name + '.txt') # 원본 PDF와 같은 폴더에 .txt로 저장

                # 이미 같은 이름의 TXT 파일이 있는지 확인 (옵션: 덮어쓸지 물어보거나 스킵)
                if os.path.exists(output_txt_path):
                    print(f"TXT 파일 이미 존재: '{os.path.basename(file)}' -> '{os.path.basename(output_txt_path)}'. 변환을 스킵합니다.")
                    txt_list.append(output_txt_path)
                    continue # 다음 파일로 넘어감

                print(f"PDF 발견: '{os.path.basename(file)}'. 텍스트 추출 및 TXT 변환 중...")
                extracted_text = extract_text_from_pdf(file_path)

                if extracted_text:
                    try:
                        # 추출된 텍스트를 TXT 파일로 저장
                        with open(output_txt_path, 'w', encoding='utf-8') as f:
                            f.write(extracted_text)
                        print(f"'{os.path.basename(file)}' -> '{os.path.basename(output_txt_path)}' 변환 및 저장 완료.")
                        txt_list.append(output_txt_path) # TXT 파일 경로 목록에 추가
                    except Exception as e:
                        print(f"TXT 파일 저장 오류 ({os.path.basename(output_txt_path)}): {e}")
                else:
                    print(f"'{os.path.basename(file)}' 텍스트 추출 실패. 변환 건너뜁니다.")

            # TXT 파일인 경우 (PDF에서 변환된 TXT 포함)
            elif file_ext == '.txt':
                print(f"TXT 파일 발견: '{os.path.basename(file)}'")
                txt_list.append(file_path) # TXT 파일 경로 목록에 추가


            # 그 외 파일 형식인 경우
            else:
                # print(f"알 수 없는 파일 형식: '{os.path.basename(file)}'. 건너뜁니다.") # 디버깅 시 출력

                pass # PDF, TXT가 아니면 무시

    print(f"\n--- Google Drive 폴더 탐색 및 파일 처리 완료. ---")

    # print("확보된 TXT 파일 경로 목록 (일부):", txt_list[:10]) # 디버깅용

    return txt_list # RAG 입력으로 사용할 TXT 파일 경로 목록 반환

In [10]:
process_drive_folder_for_rag(base_drive_folder_path)

--- Google Drive 폴더 탐색 및 파일 처리 시작: '/content/drive/MyDrive/PROJECT/cn_api_project/data' ---

폴더 탐색 중: /content/drive/MyDrive/PROJECT/cn_api_project/data

폴더 탐색 중: /content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관
TXT 파일 이미 존재: '무배당 삼성화재 다이렉트보험약관.pdf' -> '무배당 삼성화재 다이렉트보험약관.txt'. 변환을 스킵합니다.
TXT 파일 이미 존재: '삼성화재_반려묘보험_애니펫.pdf' -> '삼성화재_반려묘보험_애니펫.txt'. 변환을 스킵합니다.
TXT 파일 이미 존재: '삼성화재_반려묘보험_의기냥냥.pdf' -> '삼성화재_반려묘보험_의기냥냥.txt'. 변환을 스킵합니다.
TXT 파일 이미 존재: '한화_스마일펫보험.pdf' -> '한화_스마일펫보험.txt'. 변환을 스킵합니다.
TXT 파일 이미 존재: 'samsung_anicar_terms.pdf' -> 'samsung_anicar_terms.txt'. 변환을 스킵합니다.
TXT 파일 이미 존재: 'samsung_anicar_direct_terms.pdf' -> 'samsung_anicar_direct_terms.txt'. 변환을 스킵합니다.
TXT 파일 이미 존재: 'samsung_child_terms.pdf' -> 'samsung_child_terms.txt'. 변환을 스킵합니다.
TXT 파일 이미 존재: 'db_personal_car_terms.pdf' -> 'db_personal_car_terms.txt'. 변환을 스킵합니다.
TXT 파일

['/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/무배당 삼성화재 다이렉트보험약관.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/삼성화재_반려묘보험_애니펫.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/삼성화재_반려묘보험_의기냥냥.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/한화_스마일펫보험.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/samsung_anicar_terms.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/samsung_anicar_direct_terms.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/samsung_child_terms.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/db_personal_car_terms.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/db_cat_terms.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/db_injury_terms.txt',
 '/content/drive/MyDrive/PROJECT/cn_api_p

### 추출한 텍스트 읽고 정리

### summary template

#### summarization template

In [7]:
summarization_template = '''
## Role: 소비자 권익 보호 보험 전문가

## Task:
아래 삼성화재 실손의료보험 약관 텍스트를 읽고, 다음 지침에 따라 소비자가 이해하기 쉬운 언어로 핵심 내용을 요약하고 구조화해 주세요. 목표는 일반인이 약관의 주요 내용을 빠르고 정확하게 이해하도록 돕는 것입니다.

## 지침:
1.  **약관 구조 파악:** 제공된 약관 텍스트의 전체적인 구조(관, 조항, 항, 호 등)를 꼼꼼히 파악합니다. 특히 '제1관', '제2조', '(1)', '①', '1.' 과 같은 구조 표시를 중심으로 파악해 주세요.
2.  **목차 및 내용 요약:** 파악한 구조를 바탕으로 주요 '관', '조항', 그리고 중요한 하위 내용(항, 호, 표 등)을 구분하여 목차를 만들고 각 목차별 핵심 내용을 소비자가 이해하기 쉬운 일상적인 언어로 요약합니다.
3.  **쉽게 바꾸는 과정 설명 (CoT):** 약관 원문이 복잡하거나 전문 용어가 많아서 소비자가 이해하기 어렵다고 판단되는 **핵심적인 내용**에 대해서는, **어떻게 복잡한 원문 표현을 이해하고 어떤 과정을 거쳐 쉬운 언어로 단순화했는지 그 사고 과정을 간략히 설명**해 주세요. 이 설명은 해당 요약 바로 뒤에 `[단순화 과정]: 원문 "..."을 -> 이렇게 이해하고 -> 이렇게 쉬운 표현으로 바꿈` 형태로 포함해 주세요. 모든 요약에 포함할 필요는 없으며, 특히 중요하거나 복잡한 부분에만 적용합니다.
4.  **출력 구조화:** 목차와 요약 내용을 Markdown 형식으로 구조화하여 제시합니다.
    *   최상위 제목은 약관 이름으로 (예: `# 삼성화재 기본형 실손의료보험 약관 요약`)
    *   그 아래는 '관' 제목 (예: `## 제1관 일반사항 및 용어의 정의`)
    *   그 아래는 '조항' 제목 (예: `### 제3조 (보장종목별 보상내용)`)
    *   각 조항의 하위 내용(항, 호, 표)은 Markdown 리스트(`- `)나 하위 제목(`####`)으로 표현합니다.
    *   필요하다면 중요한 용어는 별도 섹션으로 정리해 주세요.
5.  **출력 내용:** 약관의 모든 내용을 상세히 요약하기보다, 소비자가 보험금을 청구하거나 약관을 이해하는 데 꼭 필요한 **핵심적인 내용(보상 내용, 보상하지 않는 내용, 보험금 계산 방법, 중요 조건 등)** 위주로 정리해 주세요.

## 출력 형식:
Markdown

## 분석 대상 약관 텍스트:
{clause_text}

## 출력 시작:
'''

#### modified summarization template

-> refer to the following summary content



In [8]:
summarization_template_with_context = '''
## 역할: 일반 소비자를 위한 보험 약관 통역사 (연속 작업 모드)

## 임무:
아래 **현재 분석 대상인 약관 텍스트 부분**을 읽고, **[이전 약관 내용 요약] 부분을 참고**하여 전체 약관 구조의 흐름을 놓치지 않으면서, 보험에 대해 잘 모르는 사람도 쉽고 명확하게 이해할 수 있도록 요약하고 구조화해 주세요. 이전 요약과 자연스럽게 이어지도록 Markdown 구조 레벨을 조정하고, 이미 설명된 내용은 반복하지 않거나 간략히 언급해 주세요.

[이전 약관 내용 요약]:
{previous_summary}

## 지침:
1.  **구조 파악:** 제공된 **현재 텍스트**의 주요 목차와 하위 구조를 정확히 인식합니다. **이전 요약**과 어떻게 이어지는지 고려하며 전체 약관 구조 내에서 현재 부분이 어디에 해당하는지 파악합니다.
    *   제공된 약관 텍스트의 전체적인 구조(관, 조항, 항, 호 등)를 꼼꼼히 파악합니다. 특히 '제1관', '제2조', '(1)', '①', '1.' 과 같은 구조 표시를 중심으로 파악해 주세요.
    *   파악한 구조를 바탕으로 주요 '관', '조항', 그리고 중요한 하위 내용(항, 호, 표 등)을 구분하여 목차를 만들고 각 목차별 핵심 내용을 소비자가 이해하기 쉬운 일상적인 언어로 요약합니다.
2.  **핵심 내용 추출 및 단순화:** **현재 텍스트 부분**에서 가장 중요한 정보(무엇을 보상하는지, 어떻게 계산하는지, 어떤 조건이 있는지 등)를 찾아냅니다. 약관의 어려운 전문 용어나 복잡한 표현을 볼 때마다, '이걸 초등학생도 이해하게 설명하려면?' 같은 마음으로 최대한 쉬운 일상 언어로 바꿔서 요약합니다.
3.  **용어 쉽게 설명:** 현재 텍스트에 나오는 소비자가 헷갈릴 만한 중요한 용어나 조건은 요약 내용 안에 자연스럽게 포함하여 쉽게 풀어서 설명해 줍니다. **단, 이전 요약에서 이미 자세히 설명된 용어라면 현재 요약에서는 간단히 언급만 하거나 생략해도 좋습니다.**
4.  **간결하고 명확하게:** 문장은 짧고 간결하게 작성합니다. 모호하거나 이중적인 표현은 피합니다.
5.  **쉽게 바꾸는 과정 설명 (CoT):** 약관 원문이 복잡하거나 전문 용어가 많아서 소비자가 이해하기 어렵다고 판단되는 **핵심적인 내용**에 대해서는, **어떻게 복잡한 원문 표현을 이해하고 어떤 과정을 거쳐 쉬운 언어로 단순화했는지 그 사고 과정을 간략히 설명**해 주세요. 이 설명은 해당 요약 바로 뒤에 `[단순화 과정]: 원문 "..."을 -> 이렇게 이해하고 -> 이렇게 쉬운 표현으로 바꿈` 형태로 포함해 주세요. 모든 요약에 포함할 필요는 없으며, 특히 중요하거나 복잡한 부분에만 적용합니다.
6.  **출력 구조화:** 목차와 요약 내용을 Markdown 형식으로 구조화하여 제시합니다. **이전 요약과 자연스럽게 이어지도록 적절한 Markdown 제목 레벨을 사용하세요.** (예: 이전 요약이 ##으로 끝났다면 현재 요약은 ###으로 시작하거나, 이전과 같은 레벨의 다음 목차라면 ##으로 시작)
    *   최상위 제목은 약관 이름으로 (예: `# 삼성화재 기본형 실손의료보험 약관 요약`)
    *   그 아래는 '관' 제목 (예: `## 제1관 일반사항 및 용어의 정의`)
    *   그 아래는 '조항' 제목 (예: `### 제3조 (보장종목별 보상내용)`)
    *   각 조항의 하위 내용(항, 호, 표)은 Markdown 리스트(`- `)나 하위 제목(`####`)으로 표현합니다.
    *   필요하다면 중요한 용어는 별도 섹션으로 정리해 주세요.
7.  **출력 내용:** 제공된 **현재 약관 텍스트** 내에 포함된 모든 주요 목차와 핵심 내용을 놓치지 않고 다룹니다. 약관의 모든 내용을 상세히 요약하기보다, 소비자가 보험금을 청구하거나 약관을 이해하는 데 꼭 필요한 **핵심적인 내용(보상 내용, 보상하지 않는 내용, 보험금 계산 방법, 중요 조건 등)** 위주로 정리해 주세요.


## 출력 형식:
Markdown (현재 부분 요약 내용만)

## 분석 대상 현재 약관 텍스트:
{current_clause_text}

## 출력 시작:
'''

## 요약 코드 실행
* 실행1: 3000자씩 나누어 직접 셀 10개씩 따로 요약, 이전 요약 결과를 참고하여 요약하게 함
* 실행 2: 이전 청크 요약 참고하되 요약 반복 자동화
* self-consistency 적용 코드
* 분석 text가 너무 길어서 나누어 요약하며 구조화

### **get_most_consistent_summary**

In [9]:
# summarization_template_with_context사용
# previous_summary인자 추가 및 modifying code
"""
약관 텍스트 청크에 대해 CoT 기반 쉬운 요약 프롬프트를 여러 번 실행하고
이전 요약 내용을 참고하여 가장 일관적인 요약 결과를 선택.
"""

def get_most_consistent_summary(clause_text, previous_summary="", iterations=7, model="gpt-4o-2024-05-13", temperature=0.7):
    summary_counts = defaultdict(int)
    full_summaries = {}

    # print(f"\n===== 청크 요약 Self-consistency 분석 시작 ({iterations}회 반복) =====") # 이 메시지는 외부 루프에서 출력

    target_clause_text = clause_text

    for i in range(iterations):
        # print(f"----- [{i+1}/{iterations} 번째 요약 시도 중] -----") # 반복 시도 메시지 (디버깅용)

        # 수정된 템플릿 사용 및 인자 전달
        message = summarization_template_with_context.format(
            current_clause_text=target_clause_text,
            previous_summary=previous_summary if previous_summary else "없음 (이전 내용 없음)" # 첫 청크는 빈 문자열 전달
            )

        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": message}],
                temperature=temperature,
                response_format={"type": "text"}
            ).model_dump()

            summary_content = response['choices'][0]['message']['content']

            # Markdown 코드 블록 안의 내용만 추출 시도
            match = re.search(r"```markdown\n(.*)```", summary_content, re.DOTALL)
            if match:
                clean_summary_content = match.group(1).strip()
            else:
                clean_summary_content = summary_content.strip() # 없으면 전체 사용

            # 결과가 너무 짧거나 이상하면 건너뛸 수도 있음
            if len(clean_summary_content) < 50 and iterations > 1: # 최소 글자 수 기준 (조절 필요, 반복 1회 시는 검사 X)
                # print("생성된 요약이 너무 짧거나 이상하여 스킵합니다.") # 디버깅용
                continue # 이번 결과는 무시하고 다음 반복으로

            # print(f"--- 생성된 요약 (일부) ---\n{clean_summary_content[:200].replace('\n', ' ')}...\n--------------------") # 디버깅용


            # 생성된 Markdown 텍스트 전체를 키로 사용하여 빈도 계산
            summary_counts[clean_summary_content] += 1
            # 첫 번째 나온 상세 결과 저장
            if clean_summary_content not in full_summaries:
                full_summaries[clean_summary_content] = clean_summary_content

        except Exception as e:
            print(f"API 호출 중 오류 발생: {e}")
            # 오류 발생 시 오류 메시지를 결과로 기록
            error_key = f"[API 오류: {e}]"
            summary_counts[error_key] += 1
            if error_key not in full_summaries:
                full_summaries[error_key] = error_key

    # --- Self-consistency 결과 처리 ---
    if not summary_counts:
        print("요약 결과가 없습니다.")
        return "요약 실패: 결과 없음", {} # 빈 문자열과 빈 딕셔너리 반환

    # 빈도 기준으로 정렬
    sorted_results = sorted(summary_counts.items(), key=lambda x: x[1], reverse=True)
    # for key, count in sorted_results: # 빈도표 출력은 메인 루프에서
    #      display_key = key[:100].replace('\n', ' ') + '...' if len(key) > 100 else key
    #      print(f"'{display_key}': {count}회")

    # 가장 빈도가 높은 요약 텍스트 선택
    most_frequent_summary_text = sorted_results[0][0]

    # print(f"\n--- 최적 요약 선정 (빈도 {sorted_results[0][1]}회) ---") # 최적 요약 선정 메시지 (디버깅용)
    # print(most_frequent_summary_text[:200].replace('\n', ' ') + '...') # 최적 요약 일부 출력 (디버깅용)

    return most_frequent_summary_text, full_summaries.get(most_frequent_summary_text, "상세 결과 없음")


#### 3000자씩, text가 끝날 때까지 요약 반복

In [13]:
# --- 텍스트 분할 및 요약 실행 ---
all_chunk_summaries = [] # 각 3000자 청크별 요약 결과를 저장할 리스트
start_index = 0 # 텍스트를 자르기 시작할 위치
total_text_length = len(extracted_text) if extracted_text else 0 # 전체 텍스트 길이 확인 (None 또는 빈 문자열 방지)
chunk_count = 0 # 몇 번째 청크인지 기록

NameError: name 'extracted_text' is not defined

### 실행1
> 3000자씩 나누어 반복 요약, 이전 요약 결과 참고하며 요약 반복 -> 셀 10개

#### split the text and set code block

In [10]:
# --- 설정 ---
chunk_size = 2000 # 한 번에 요약할 텍스트의 문자 수
summary_iterations = 5 # 각 청크별 Self-consistency 반복 횟수
gpt_model = "gpt-4o-2024-05-13" # 요약에 사용할 GPT 모델
gpt_temperature = 0.7 # 요약 내용 다양성을 위한 온도 설정

In [8]:
num_chunks = 10 # 나눌 청크 개수
total_text_length = len(extracted_text) if extracted_text else 0

if total_text_length == 0:
    print("오류: extracted_text 변수가 비어있습니다. 약관 전체 텍스트를 넣어주세요.")
else:
    print(f"--- 전체 텍스트 길이: {total_text_length} 문자 ---")
# 각 청크의 대략적인 크기 계산
base_chunk_size = total_text_length // num_chunks
remainder = total_text_length % num_chunks

NameError: name 'extracted_text' is not defined

In [11]:
# 각 청크의 시작 및 끝 인덱스 계산
chunk_indices = []
current_start = 0
for i in range(num_chunks):
    current_end = current_start + base_chunk_size
    if i < remainder: # 나머지 글자는 앞쪽 청크에 1자씩 추가
        current_end += 1
    chunk_indices.append((current_start, current_end))
    current_start = current_end

print(f"--- 전체 텍스트 ({total_text_length} 자)를 {num_chunks} 개의 청크로 분할 ---")
# print(f"각 청크 인덱스: {chunk_indices}") # 디버깅용

NameError: name 'num_chunks' is not defined

#### 함수 실행
> summary with 10 code chunks divided

In [ ]:
start = time.time()
# summarization with chunk 1
# get 1st chunk index
start_index, end_index = chunk_indices[0]
# 약관 텍스트 청크 추출
clause_chunk_1 = extracted_text[start_index:end_index]

print(f"\n===== [청크 1/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 첫 번째 청크는 이전 요약이 없으므로 previous_summary는 빈 문자열('')
summary_chunk_1, _ = get_most_consistent_summary(
    clause_text=clause_chunk_1,
    previous_summary="", # 이전 요약 없음
    iterations=summary_iterations, # 셀 1에서 설정한 반복 횟수 사용
    model=gpt_model, # 셀 1에서 설정한 모델 사용
    temperature=gpt_temperature # 셀 1에서 설정한 온도 사용
)

print(f"\n--- 청크 1/10 요약 완료. 결과가 summary_chunk_1 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
start = time.time()
# summary 2
start_index, end_index = chunk_indices[1]

# 약관 텍스트 청크 추출
clause_chunk_2 = extracted_text[start_index:end_index]

print(f"\n===== [청크 2/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_1를 previous_summary로 전달
summary_chunk_2, _ = get_most_consistent_summary(
    clause_text=clause_chunk_2,
    previous_summary=summary_chunk_1, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 2/10 요약 완료. 결과가 summary_chunk_2 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 3
start_index, end_index = chunk_indices[2]

# 약관 텍스트 청크 추출
clause_chunk_3 = extracted_text[start_index:end_index]

print(f"\n===== [청크 3/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_2를 previous_summary로 전달
summary_chunk_3, _ = get_most_consistent_summary(
    clause_text=clause_chunk_3,
    previous_summary=summary_chunk_2, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 3/10 요약 완료. 결과가 summary_chunk_3 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 4
start_index, end_index = chunk_indices[3]

# 약관 텍스트 청크 추출
clause_chunk_4 = extracted_text[start_index:end_index]

print(f"\n===== [청크 4/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_3을 previous_summary로 전달
summary_chunk_4, _ = get_most_consistent_summary(
    clause_text=clause_chunk_4,
    previous_summary=summary_chunk_3, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 4/10 요약 완료. 결과가 summary_chunk_4 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 5
start_index, end_index = chunk_indices[4]

# 약관 텍스트 청크 추출
clause_chunk_5 = extracted_text[start_index:end_index]

print(f"\n===== [청크 5/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_4를 previous_summary로 전달
summary_chunk_5, _ = get_most_consistent_summary(
    clause_text=clause_chunk_5,
    previous_summary=summary_chunk_4, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 5/10 요약 완료. 결과가 summary_chunk_5 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 6
start_index, end_index = chunk_indices[5]

# 약관 텍스트 청크 추출
clause_chunk_6 = extracted_text[start_index:end_index]

print(f"\n===== [청크 6/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_5를 previous_summary로 전달
summary_chunk_6, _ = get_most_consistent_summary(
    clause_text=clause_chunk_6,
    previous_summary=summary_chunk_5, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 6/10 요약 완료. 결과가 summary_chunk_6 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 7
start_index, end_index = chunk_indices[6]

# 약관 텍스트 청크 추출
clause_chunk_7 = extracted_text[start_index:end_index]

print(f"\n===== [청크 7/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_6를 previous_summary로 전달
summary_chunk_7, _ = get_most_consistent_summary(
    clause_text=clause_chunk_7,
    previous_summary=summary_chunk_6, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 7/10 요약 완료. 결과가 summary_chunk_7 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 8
start_index, end_index = chunk_indices[7]

# 약관 텍스트 청크 추출
clause_chunk_8 = extracted_text[start_index:end_index]

print(f"\n===== [청크 8/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_7를 previous_summary로 전달
summary_chunk_8, _ = get_most_consistent_summary(
    clause_text=clause_chunk_8,
    previous_summary=summary_chunk_7, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 8/10 요약 완료. 결과가 summary_chunk_8 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 9
start_index, end_index = chunk_indices[8]

# 약관 텍스트 청크 추출
clause_chunk_9 = extracted_text[start_index:end_index]

print(f"\n===== [청크 9/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_9를 previous_summary로 전달
summary_chunk_9, _ = get_most_consistent_summary(
    clause_text=clause_chunk_9,
    previous_summary=summary_chunk_8, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 9/10 요약 완료. 결과가 summary_chunk_9 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
start = time.time()
# summary 10
start_index, end_index = chunk_indices[9]

# 약관 텍스트 청크 추출
clause_chunk_10 = extracted_text[start_index:end_index]

print(f"\n===== [청크 10/10] {start_index} 부터 {end_index} 까지 요약 시작 =====")
# get_most_consistent_summary 함수를 사용하여 현재 청크 요약
# 이전 청크 요약 결과인 summary_chunk_9를 previous_summary로 전달
summary_chunk_10, _ = get_most_consistent_summary(
    clause_text=clause_chunk_10,
    previous_summary=summary_chunk_9, # 이전 요약 결과 전달!
    iterations=summary_iterations,
    model=gpt_model,
    temperature=gpt_temperature
)

print(f"\n--- 청크 10/10 요약 완료. 결과가 summary_chunk_10 변수에 저장되었습니다. ---")
end = time.time()
print(f"--- {end - start} 초 걸림 ---")

In [ ]:
print(summary_chunk_1)

In [ ]:
# 각 결과를 각각 text파일로 만들
for i in range(10):

    chunk_number = i + 1
    current_text = f"summary_chunk_{chunk_number}"
    file_name = f"summary_chunk_{chunk_number}.txt"
    # 전체 파일 경로 생성 (저장 폴더 경로 + 파일 이름)
    full_output_path = os.path.join('/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt', file_name)
    print(f"💾 저장 중: {full_output_path}")
    try:
        with open(full_output_path, "w", encoding="utf-8") as file:
            file.write(current_text)
            print(f"{file_name} 파일을 읽었습니다.")
    except FileNotFoundError:
        print(f"{file_name} 파일을 찾을 수 없습니다.")

In [ ]:
# 최종 약관 요약 결과 합치기
# 이전 요약 셀 실행 이후 실행

print("\n\n===== 최종 전체 약관 요약 결과 합치는 중 =====")

# 10개의 청크 요약 결과를 하나의 변수에 순서대로 합칩니다.
# 각 변수가 존재하고 비어있지 않은 경우에만 합칩니다.
full_extracted_summary = ""

# summary_chunk_1 부터 summary_chunk_10 까지 순서대로 합칩니다.
# 각 청크 요약 사이에 줄바꿈을 추가하여 구분합니다.

if 'summary_chunk_1' in locals() and summary_chunk_1: # 변수 존재 및 내용 확인
    full_extracted_summary += summary_chunk_1 + "\n\n"

if 'summary_chunk_2' in locals() and summary_chunk_2:
    full_extracted_summary += summary_chunk_2 + "\n\n"

if 'summary_chunk_3' in locals() and summary_chunk_3:
    full_extracted_summary += summary_chunk_3 + "\n\n"

if 'summary_chunk_4' in locals() and summary_chunk_4:
    full_extracted_summary += summary_chunk_4 + "\n\n"

if 'summary_chunk_5' in locals() and summary_chunk_5:
    full_extracted_summary += summary_chunk_5 + "\n\n"

if 'summary_chunk_6' in locals() and summary_chunk_6:
    full_extracted_summary += summary_chunk_6 + "\n\n"

if 'summary_chunk_7' in locals() and summary_chunk_7:
    full_extracted_summary += summary_chunk_7 + "\n\n"

if 'summary_chunk_8' in locals() and summary_chunk_8:
    full_extracted_summary += summary_chunk_8 + "\n\n"

if 'summary_chunk_9' in locals() and summary_chunk_9:
    full_extracted_summary += summary_chunk_9 + "\n\n"

if 'summary_chunk_10' in locals() and summary_chunk_10:
    # 마지막 청크 요약이므로 뒤에 줄바꿈을 과도하게 넣지 않거나 조절
    full_extracted_summary += summary_chunk_10

print("\n\n===== 최종 전체 약관 요약 결과 (full_extracted_summary 변수) =====")
# 최종 결과 출력
print(full_extracted_summary)

print("\n--- 최종 결과 합치기 완료. full_extracted_summary 변수를 활용하세요. ---")

In [ ]:
current_text = full_extracted_summary
file_name = "full_summary.txt"
# 전체 파일 경로 생성 (저장 폴더 경로 + 파일 이름)
file_full_path = os.path.join('/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt', file_name)
print(f"💾 저장 중: {file_full_path}")
try:
    with open(file_full_path, "w", encoding="utf-8") as file:
        file.write(current_text)
        print(f"{file_name} 파일을 읽었습니다.")
except FileNotFoundError:
    print(f"{file_name} 파일을 찾을 수 없습니다.")

### 실행2

#### summarize_policy_in_chunks 함수 생성

> 긴 약관 텍스트를 지정된 크기로 나누고, 각 청크별로 Self-consistency 요약을 수행하며,
이전 청크의 요약 결과를 다음 청크 요약 시 참고하여 결과를 모아 반환


In [12]:
import time

In [13]:
# --- 자동화된 약관 청크별 요약 함수 ---
def summarize_policy_in_chunks(
    extracted_text,
    chunk_size=2000, # 요약할 텍스트 청크 크기
    summary_iterations=5, # 각 청크별 Self-consistency 반복 횟수
    model="gpt-4o-2024-05-13", # 요약에 사용할 GPT 모델
    temperature=0.7 # 요약 내용 다양성을 위한 온도 설정
):
    if not extracted_text:
        print("오류: 요약할 텍스트가 비어있습니다.")
        return []

    if 'get_most_consistent_summary' not in globals():
        print("오류: get_most_consistent_summary 함수가 정의되지 않았습니다.")
        return []

    if client is None:
        print("오류: OpenAI API 클라이언트가 초기화되지 않았습니다.")
        return []

    all_chunk_summaries = [] # 각 청크별 요약 결과를 저장할 리스트
    start_index = 0 # 텍스트를 자르기 시작할 위치
    total_text_length = len(extracted_text)
    chunk_count = 0 # 몇 번째 청크인지 기록
    previous_summary = "" # 이전 청크의 요약 결과를 저장할 변수 (초기값 빈 문자열)

    print(f"--- 전체 텍스트 길이: {total_text_length} 문자 ---")
    print(f"--- {chunk_size} 문자 단위로 분할하여 요약 진행 ---")

    start_time_total = time.time() # 전체 요약 시작 시간

    # extracted_text가 끝날 때까지 반복
    while start_index < total_text_length:
        start_time_chunk = time.time() # 청크 요약 시작 시간

        # 현재 청크의 끝 위치 계산 (전체 텍스트 길이를 넘지 않도록 min 함수 사용)
        end_index = min(start_index + chunk_size, total_text_length)

        # 현재 요약할 텍스트 청크 추출
        current_chunk = extracted_text[start_index:end_index]

        # 추출된 청크가 비어있거나 공백만 있는지 확인
        if not current_chunk.strip():
            print(f"청크 {chunk_count + 1}: 내용이 비어있거나 공백만 있습니다. 처리를 종료합니다.")
            break # 내용이 없으면 더 이상 처리할 청크가 없으므로 루프 종료

        chunk_count += 1
        print(f"\n\n===== [청크 {chunk_count}] {start_index} 부터 {end_index} 까지 요약 시작 =====")

        # 정의된 get_most_consistent_summary 함수를 사용하여 현재 청크 요약
        # 이전 청크 요약 결과 (previous_summary)를 인자로 전달합니다.
        consistent_summary_text, _ = get_most_consistent_summary(
            clause_text=current_chunk,
            previous_summary=previous_summary, # <-- 이전 청크 요약 전달!
            iterations=summary_iterations,
            model=model,
            temperature=temperature
        )

        # 요약 결과를 리스트에 추가
        all_chunk_summaries.append(consistent_summary_text)

        # 현재 청크의 요약 결과를 다음 청크의 '이전 요약'으로 업데이트
        previous_summary = consistent_summary_text

        # 다음 청크의 시작 위치 업데이트
        start_index = end_index

        end_time_chunk = time.time() # 청크 요약 종료 시간
        print(f"\n--- 청크 {chunk_count} 요약 완료. ({end_time_chunk - start_time_chunk:.2f} 초 걸림) ---")


    end_time_total = time.time() # 전체 요약 종료 시간
    print(f"\n--- 전체 텍스트 요약 완료. 총 {chunk_count} 개의 청크 요약 생성 ({end_time_total - start_time_total:.2f} 초 걸림) ---")

    combined_summary = "\n\n".join(all_chunk_summaries)
    # return combined_summary, all_chunk_summaries

    # 여기서는 청크별 요약 리스트만 반환합니다. 필요시 외부에서 합칩니다.
    return all_chunk_summaries

In [ ]:
with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/db_cat_terms.txt', 'r', encoding = 'utf-8') as f:
    db_cat_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/db_injury_terms.txt', 'r', encoding = 'utf-8') as f:
    db_injury_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/db_teeth_terms.txt', 'r', encoding = 'utf-8') as f:
    db_teeth_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/컴퓨터네트워크/cn_api_project/data/보험약관/삼성화재_반려묘보험_의기냥냥.txt', 'r', encoding = 'utf-8') as f:
    samsung_nyang_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/컴퓨터네트워크/cn_api_project/data/보험약관/한화_스마일펫보험.txt', 'r', encoding = 'utf-8') as f:
    hanhwa_pet_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/컴퓨터네트워크/cn_api_project/data/보험약관/삼성화재_반려묘보험_애니펫.txt', 'r', encoding = 'utf-8') as f:
    samsung_cat_anipet_terms = f.read()

In [14]:
with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/메리츠 마음든든 반려동물보험약관.txt', 'r', encoding = 'utf-8') as f:
    메리츠_반려동물보험약관_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/메리츠 반려동물보험약관.txt', 'r', encoding= 'utf-8') as f:
    메리츠_마음든든_반려동물보험약관_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/KB_cat_direct_terms.txt', 'r', encoding= 'utf-8') as f:
    KB_다이렉트_반려묘보험_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/hyundai_cat_direct_terms.txt', 'r', encoding = 'utf-8') as f:
    hyundai_다이렉트_반려묘보험_terms = f.read()

with open('/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/meritz_direct_cat_terms.txt', 'r', encoding = 'utf-8') as f:
    메리츠_다이렉트_펫퍼민트_반려묘보험_terms = f.read()



In [19]:
original_filename = 'db_cat_terms.txt'
original_filename_ext, ext = os.path.splitext(original_filename)
original_filename_without_ext = original_filename_ext
summary_filename = f'{original_filename_ext}_summary.txt'
save_folder = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary'
summary_path = os.path.join(save_folder, summary_filename)

print(summary_filename, summary_path)

db_cat_terms_summary.txt /content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary/db_cat_terms_summary.txt


In [22]:
def summary_path_generator(original_filename):
    original_filename_ext, ext = os.path.splitext(original_filename)
    original_filename_without_ext = original_filename_ext
    summary_filename = f'{original_filename_ext}_summary.txt'
    return summary_filename

In [17]:
def save_summary_txt(summary_text, summary_path):
    try:
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write(summary_text)
    except:
        print(f"파일 저장 오류: {summary_path}")

In [23]:
extracted_text = hyundai_다이렉트_반려묘보험_terms
original_filename = 'hyundai_다이렉트_반려묘보험_terms.txt'
save_folder = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary'
summary_path = os.path.join(save_folder, summary_path_generator(original_filename))
print(summary_path)

/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary/hyundai_다이렉트_반려묘보험_terms_summary.txt


In [24]:
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=3000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력
        save_summary_txt(full_extracted_summary_combined, summary_path)

    else:
        print("생성된 요약 결과가 없습니다.")



===== 약관 청크별 자동 요약 프로세스 시작 =====
--- 전체 텍스트 길이: 469727 문자 ---
--- 3000 문자 단위로 분할하여 요약 진행 ---


===== [청크 1] 0 부터 3000 까지 요약 시작 =====

--- 청크 1 요약 완료. (19.96 초 걸림) ---


===== [청크 2] 3000 부터 6000 까지 요약 시작 =====

--- 청크 2 요약 완료. (28.47 초 걸림) ---


===== [청크 3] 6000 부터 9000 까지 요약 시작 =====

--- 청크 3 요약 완료. (35.80 초 걸림) ---


===== [청크 4] 9000 부터 12000 까지 요약 시작 =====

--- 청크 4 요약 완료. (38.27 초 걸림) ---


===== [청크 5] 12000 부터 15000 까지 요약 시작 =====

--- 청크 5 요약 완료. (31.00 초 걸림) ---


===== [청크 6] 15000 부터 18000 까지 요약 시작 =====

--- 청크 6 요약 완료. (25.77 초 걸림) ---


===== [청크 7] 18000 부터 21000 까지 요약 시작 =====

--- 청크 7 요약 완료. (42.25 초 걸림) ---


===== [청크 8] 21000 부터 24000 까지 요약 시작 =====

--- 청크 8 요약 완료. (47.74 초 걸림) ---


===== [청크 9] 24000 부터 27000 까지 요약 시작 =====

--- 청크 9 요약 완료. (46.01 초 걸림) ---


===== [청크 10] 27000 부터 30000 까지 요약 시작 =====

--- 청크 10 요약 완료. (36.51 초 걸림) ---


===== [청크 11] 30000 부터 33000 까지 요약 시작 =====

--- 청크 11 요약 완료. (43.30 초 걸림) ---


===== [청크 12] 33000 부터 36000 까지 요약 시작 ==

KeyboardInterrupt: 

#### 청크로 나누고 요약하는 함수 실행

- db손해보험-반려묘 보험

In [ ]:
extracted_text = db_cat_terms
save_path = '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt/summary/db_cat_terms_summary.txt'

In [ ]:
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=3000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력

        # 필요하다면 이 합쳐진 요약 결과를 파일로 저장할 수 있습니다.
        try:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(full_extracted_summary_combined)
            print(f"\n전체 요약 결과 파일 저장 완료: {save_path}")
        except Exception as e: print(f"파일 저장 오류: {e}")

    else:
        print("생성된 요약 결과가 없습니다.")

else:
    print("\n\n===== 자동 요약 프로세스 실행 불가 =====")
    if not extracted_text: print("요약할 extracted_text가 비어있습니다.")
    if client is None: print("OpenAI API 클라이언트가 초기화되지 않았습니다.")
    if 'get_most_consistent_summary' not in globals(): print("get_most_consistent_summary 함수가 정의되지 않았습니다.")


* 삼성생명 의기냥냥 펫보험

In [ ]:
extracted_text = samsung_nyang_terms
save_path = '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt/summary/samsung_nyang_terms_summary.txt'

In [ ]:
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=2000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력

        # 필요하다면 이 합쳐진 요약 결과를 파일로 저장할 수 있습니다.
        try:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(full_extracted_summary_combined)
            print(f"\n전체 요약 결과 파일 저장 완료: {save_path}")
        except Exception as e: print(f"파일 저장 오류: {e}")

    else:
        print("생성된 요약 결과가 없습니다.")

else:
    print("\n\n===== 자동 요약 프로세스 실행 불가 =====")
    if not extracted_text: print("요약할 extracted_text가 비어있습니다.")
    if client is None: print("OpenAI API 클라이언트가 초기화되지 않았습니다.")
    if 'get_most_consistent_summary' not in globals(): print("get_most_consistent_summary 함수가 정의되지 않았습니다.")

In [23]:
extracted_text = 메리츠_반려동물보험약관_terms
save_path = '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt/summary/메리츠_반려동물보험약관_terms_summary.txt'

In [ ]:
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=3000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력

        # 필요하다면 이 합쳐진 요약 결과를 파일로 저장할 수 있습니다.
        try:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(full_extracted_summary_combined)
            print(f"\n전체 요약 결과 파일 저장 완료: {save_path}")
        except Exception as e: print(f"파일 저장 오류: {e}")

    else:
        print("생성된 요약 결과가 없습니다.")

else:
    print("\n\n===== 자동 요약 프로세스 실행 불가 =====")
    if not extracted_text: print("요약할 extracted_text가 비어있습니다.")
    if client is None: print("OpenAI API 클라이언트가 초기화되지 않았습니다.")
    if 'get_most_consistent_summary' not in globals(): print("get_most_consistent_summary 함수가 정의되지 않았습니다.")



===== 약관 청크별 자동 요약 프로세스 시작 =====
--- 전체 텍스트 길이: 95178 문자 ---
--- 3000 문자 단위로 분할하여 요약 진행 ---


===== [청크 1] 0 부터 3000 까지 요약 시작 =====

--- 청크 1 요약 완료. (24.88 초 걸림) ---


===== [청크 2] 3000 부터 6000 까지 요약 시작 =====

--- 청크 2 요약 완료. (21.06 초 걸림) ---


===== [청크 3] 6000 부터 9000 까지 요약 시작 =====

--- 청크 3 요약 완료. (23.50 초 걸림) ---


===== [청크 4] 9000 부터 12000 까지 요약 시작 =====

--- 청크 4 요약 완료. (16.28 초 걸림) ---


===== [청크 5] 12000 부터 15000 까지 요약 시작 =====

--- 청크 5 요약 완료. (23.98 초 걸림) ---


===== [청크 6] 15000 부터 18000 까지 요약 시작 =====

--- 청크 6 요약 완료. (22.98 초 걸림) ---


===== [청크 7] 18000 부터 21000 까지 요약 시작 =====

--- 청크 7 요약 완료. (14.90 초 걸림) ---


===== [청크 8] 21000 부터 24000 까지 요약 시작 =====

--- 청크 8 요약 완료. (22.32 초 걸림) ---


===== [청크 9] 24000 부터 27000 까지 요약 시작 =====

--- 청크 9 요약 완료. (28.31 초 걸림) ---


===== [청크 10] 27000 부터 30000 까지 요약 시작 =====


* 삼성생명 애니펫 반려묘보험(펫보험)

In [21]:
extracted_text = samsung_cat_anipet_terms
save_path = '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt/summary/samsung_cat_anipet_terms_summary.txt'

NameError: name 'samsung_cat_anipet_terms' is not defined

In [ ]:
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=3000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력

        # 필요하다면 이 합쳐진 요약 결과를 파일로 저장할 수 있습니다.
        try:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(full_extracted_summary_combined)
            print(f"\n전체 요약 결과 파일 저장 완료: {save_path}")
        except Exception as e: print(f"파일 저장 오류: {e}")

    else:
        print("생성된 요약 결과가 없습니다.")

else:
    print("\n\n===== 자동 요약 프로세스 실행 불가 =====")
    if not extracted_text: print("요약할 extracted_text가 비어있습니다.")
    if client is None: print("OpenAI API 클라이언트가 초기화되지 않았습니다.")
    if 'get_most_consistent_summary' not in globals(): print("get_most_consistent_summary 함수가 정의되지 않았습니다.")

* 한화 스마일펫 펫보험

In [ ]:
extracted_text = hanhwa_pet_terms
save_path = '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt/summary/hanhwa_pet_terms_summary.txt'

In [ ]:
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=3000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력

        # 필요하다면 이 합쳐진 요약 결과를 파일로 저장할 수 있습니다.
        try:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(full_extracted_summary_combined)
            print(f"\n전체 요약 결과 파일 저장 완료: {save_path}")
        except Exception as e: print(f"파일 저장 오류: {e}")

    else:
        print("생성된 요약 결과가 없습니다.")

else:
    print("\n\n===== 자동 요약 프로세스 실행 불가 =====")
    if not extracted_text: print("요약할 extracted_text가 비어있습니다.")
    if client is None: print("OpenAI API 클라이언트가 초기화되지 않았습니다.")
    if 'get_most_consistent_summary' not in globals(): print("get_most_consistent_summary 함수가 정의되지 않았습니다.")

* 메리츠 마음든든 펫보험

In [ ]:
extracted_text = meritz_cat_1_terms
save_path = '/content/drive/MyDrive/UNIV./25-1/컴퓨터네트워크/cn_api_project/txt/summary/meritz_cat_1_terms.txt'

In [ ]:
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=2000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력

        # 필요하다면 이 합쳐진 요약 결과를 파일로 저장할 수 있습니다.
        try:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(full_extracted_summary_combined)
            print(f"\n전체 요약 결과 파일 저장 완료: {save_path}")
        except Exception as e: print(f"파일 저장 오류: {e}")

    else:
        print("생성된 요약 결과가 없습니다.")

else:
    print("\n\n===== 자동 요약 프로세스 실행 불가 =====")
    if not extracted_text: print("요약할 extracted_text가 비어있습니다.")
    if client is None: print("OpenAI API 클라이언트가 초기화되지 않았습니다.")
    if 'get_most_consistent_summary' not in globals(): print("get_most_consistent_summary 함수가 정의되지 않았습니다.")

* 메리츠 반려동물 보험약관

In [19]:
extracted_text = meritz_cat_2_terms
save_path = '/content/drive/MyDrive/PROJECT/cn_api_project/txt/summary/meritz_cat_2_terms.txt'

NameError: name 'meritz_cat_2_terms' is not defined

In [ ]:
start = time.time()
if extracted_text and client is not None and 'get_most_consistent_summary' in globals():
    print("\n\n===== 약관 청크별 자동 요약 프로세스 시작 =====")

    # 함수 호출 및 결과 받기
    chunk_summaries_list = summarize_policy_in_chunks(
        extracted_text=extracted_text,
        chunk_size=2000, # 3000자씩 나누어 요약
        summary_iterations=3, # Self-consistency 7회 반복
        model="gpt-4o-2024-05-13", # 사용할 GPT 모델
        temperature=0.7 # 발상 온도
    )

    print("\n===== 약관 청크별 자동 요약 프로세스 완료 =====")

    # 결과 활용 (예: 모든 요약 결과를 합쳐서 출력 또는 파일 저장)
    if chunk_summaries_list:
        print(f"총 {len(chunk_summaries_list)}개의 청크 요약 결과가 생성되었습니다.")
        # 모든 청크 요약을 하나의 문자열로 합치기
        full_extracted_summary_combined = "\n\n".join(chunk_summaries_list)

        print("\n--- 전체 요약 결과 (합본) ---")
        print(full_extracted_summary_combined) # 합쳐진 요약 결과 출력

        # 필요하다면 이 합쳐진 요약 결과를 파일로 저장할 수 있습니다.
        try:
            with open(save_path, 'w', encoding='utf-8') as f:
                f.write(full_extracted_summary_combined)
            print(f"\n전체 요약 결과 파일 저장 완료: {save_path}")
        except Exception as e: print(f"파일 저장 오류: {e}")

    else:
        print("생성된 요약 결과가 없습니다.")

else:
    print("\n\n===== 자동 요약 프로세스 실행 불가 =====")
    if not extracted_text: print("요약할 extracted_text가 비어있습니다.")
    if client is None: print("OpenAI API 클라이언트가 초기화되지 않았습니다.")
    if 'get_most_consistent_summary' not in globals(): print("get_most_consistent_summary 함수가 정의되지 않았습니다.")

end = time.time()
print(f'({end}-{start})초 걸림')

#### save folder -> data/보험약관 폴더를 순회하며 보험약관 요약하기

In [15]:
# --- 💡 사용자 경로 (원본 및 저장 경로) ---
# 요약할 원본 .txt 파일들이 있는 폴더 경로
source_folder = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/' # 예시 경로
# 요약 결과를 저장할 폴더 경로
save_folder = '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary' # 예시 경로

# --- 📦 필요한 함수 및 객체 (미리 정의되어 있어야 함) ---
# 예: summarize_policy_in_chunks 함수, get_most_consistent_summary 함수, client 객체

# --- ✅ 요약 결과 저장 폴더 생성 (만약 없다면) ---
# 요약 결과를 저장할 폴더가 없다면 미리 생성합니다.
if not os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)
    print(f"저장 폴더 생성: '{save_folder}'")
else:
    print(f"저장 폴더 확인: '{save_folder}'")


# --- 🔁 디렉토리 내 모든 .txt 파일 순회 및 조건부 요약 실행 ---
print(f"\n--- 폴더 내 .txt 파일 순회 및 요약 시작: '{source_folder}' ---")

# os.listdir()은 폴더 안의 파일/폴더 이름 리스트만 반환합니다.
# 하위 폴더까지 탐색하려면 os.walk()를 사용해야 합니다.
# 여기서는 일단 source_folder 바로 아래의 .txt 파일만 대상으로 합니다.
# 하위 폴더까지 포함하려면 이 루프를 os.walk() 기반으로 변경해야 합니다.

for filename in os.listdir(source_folder):
    # 파일 이름이 .txt로 끝나는지 확인 (확장자는 소문자로 변환 후 비교)
    if filename.lower().endswith('.txt'):
        file_path = os.path.join(source_folder, filename) # 원본 .txt 파일 전체 경로

        # 요약 결과를 저장할 파일 이름 만들기
        # 원본 파일 이름에서 확장자(.txt)를 제거하고 '_summary.txt'를 붙입니다.
        name_part = os.path.splitext(filename)[0] # 확장자 제거
        summary_filename = f"{name_part}_summary.txt" # 예: "원본파일_summary.txt"
        save_path = os.path.join(save_folder, summary_filename) # 저장될 요약 파일의 전체 경로


        # --- !!! 요약 파일 존재 여부 확인 및 조건부 요약 실행 !!! ---
        if os.path.exists(save_path):
            # 요약 파일이 이미 존재하면 요약 과정을 건너뜁니다.
            print(f"\n\n✅ 이미 요약된 파일 존재: '{summary_filename}'. 요약 건너뜁니다.")
            continue # 현재 파일 처리를 중단하고 다음 파일로 넘어갑니다.
        else:
            # 요약 파일이 존재하지 않으면 요약을 진행합니다.
            print(f"\n\n📄 파일 요약 시작: {filename}")

            try:
                # 원본 .txt 파일 내용 읽기
                with open(file_path, 'r', encoding='utf-8') as f:
                    extracted_text = f.read()

                # --- 약관 요약 실행 ---
                # summarize_policy_in_chunks 함수가 정의되어 있다고 가정합니다.
                # 이 함수 내부에서 get_most_consistent_summary가 호출됩니다.
                print("  - 요약 함수 호출 중...")
                chunk_summaries = summarize_policy_in_chunks(
                    extracted_text,
                    chunk_size=2000, # 요약 청크 크기 (조절 가능)
                    summary_iterations=5, # Self-consistency 반복 횟수 (조절 가능)
                    model="gpt-4o-2024-05-13",
                    temperature=0.7
                )
                print("  - 요약 함수 실행 완료.")


                # --- 요약 결과 저장 ---
                # 요약 파일이 존재하지 않으므로 저장합니다.
                # save_path 변수는 이미 위에서 만들어졌습니다.
                print(f"  - 요약 결과 저장 중: '{summary_filename}'")
                try:
                    # 파일을 쓰기 모드('w')로 열고 요약 결과 쓰기
                    with open(save_path, 'w', encoding='utf-8') as out_file:
                        # chunk_summaries는 요약된 각 청크 문자열의 리스트입니다.
                        for i, chunk in enumerate(chunk_summaries, start=1):
                            out_file.write(f"### 청크 {i} 요약 ###\n{chunk}\n\n") # 각 청크 요약 결과 쓰기

                    print(f"✅ 저장 완료: '{summary_filename}'")

                except IOError as e:
                    print(f"❌ 파일 저장 오류 ({summary_filename}): {e}")
                    print(f"경로 확인: '{save_path}' 경로에 쓸 수 있는지, Google Drive 마운트 상태가 올바른지 확인하세요.")

            except Exception as e:
                # 파일 읽기 또는 요약 실행 중 발생한 다른 오류 처리
                print(f"❌ 파일 처리 중 오류 발생 ({filename}): {e}")

print("\n--- 모든 .txt 파일 요약 프로세스 완료 ---")


저장 폴더 확인: '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/summary'

--- 폴더 내 .txt 파일 순회 및 요약 시작: '/content/drive/MyDrive/PROJECT/cn_api_project/data/보험약관/' ---


✅ 이미 요약된 파일 존재: '무배당 삼성화재 다이렉트보험약관_summary.txt'. 요약 건너뜁니다.


✅ 이미 요약된 파일 존재: '삼성화재_반려묘보험_애니펫_summary.txt'. 요약 건너뜁니다.


✅ 이미 요약된 파일 존재: '삼성화재_반려묘보험_의기냥냥_summary.txt'. 요약 건너뜁니다.


✅ 이미 요약된 파일 존재: '한화_스마일펫보험_summary.txt'. 요약 건너뜁니다.


✅ 이미 요약된 파일 존재: 'samsung_anicar_terms_summary.txt'. 요약 건너뜁니다.


✅ 이미 요약된 파일 존재: 'samsung_anicar_direct_terms_summary.txt'. 요약 건너뜁니다.


📄 파일 요약 시작: samsung_child_terms.txt
  - 요약 함수 호출 중...
--- 전체 텍스트 길이: 2023225 문자 ---
--- 2000 문자 단위로 분할하여 요약 진행 ---


===== [청크 1] 0 부터 2000 까지 요약 시작 =====

--- 청크 1 요약 완료. (28.64 초 걸림) ---


===== [청크 2] 2000 부터 4000 까지 요약 시작 =====

--- 청크 2 요약 완료. (30.87 초 걸림) ---


===== [청크 3] 4000 부터 6000 까지 요약 시작 =====

--- 청크 3 요약 완료. (28.30 초 걸림) ---


===== [청크 4] 6000 부터 8000 까지 요약 시작 =====


KeyboardInterrupt: 